In [5]:
import random
import numpy as np
from statsmodels.tsa.arima_process import arma_generate_sample
import matplotlib.pyplot as plt
import pandas as pd 
import hts
from hts.hierarchy import HierarchyTree

from  forecasting_functions import probabilistic_forecasting
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from darts.utils.missing_values import fill_missing_values
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import TCNModel,StatsForecastAutoARIMA,NHiTSModel
import scipy.stats as ss
from darts.utils.likelihood_models import GaussianLikelihood

import properscoring as ps

# Fitting ARIMA model manually 
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt
from sklearn.metrics import mean_squared_error
import itertools 



In [6]:
''' pl follow steps from: https://github.com/time-series-foundation-models/lag-llama/blob/main/README.md 
to set the working directory '''

' pl follow steps from: https://github.com/time-series-foundation-models/lag-llama/blob/main/README.md \nto set the working directory '

In [7]:
df = pd.read_csv("datasets/tourism.csv")

try:
    test_len = int(input("Please enter the test length as an integer: "))
    print(f"Test length is set to: {test_len}")
except ValueError:
    print("Invalid input. Please enter a valid integer.")


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/tourism.csv'

In [ ]:
cd /lag-llama/


In [ ]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir  lag-llama

In [ ]:
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

In [ ]:
def get_lag_llama_predictions(dataset, prediction_length, context_length=32, num_samples=20, device="cuda", batch_size=64, nonnegative_pred_samples=True):
    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],

        nonnegative_pred_samples=nonnegative_pred_samples,

        # linear positional encoding scaling
        rope_scaling={
            "type": "linear",
            "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        },

        batch_size=batch_size,
        num_parallel_samples=num_samples,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))
    tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))

    return forecasts, tss

In [ ]:

''' Data consist of the information of all the tourists who visited Australia for various purposes between January 1998 to October 2006
      Data is available every quarter. Data is mainly disaggregated based on four measure purposes of travel: Holiday (Hol), Vis-
iting Friends and Relatives (VFR), Business (Bus), and Other (Oth). There are a total
of four levels in the hierarchy. The total number of the series in the dataset is 89.


Reference 
---------------

1. Kamarthi, Harshavardhan, et al. "PROFHIT: Probabilistic Robust Forecasting for Hierarchical Time-series.
" arXiv preprint arXiv:2206.07940 (2022).

      
      '''


df.rename(columns = {'Total':'total', 'Hol':'hol','VFR':'vfr','Bus':'bus','Oth':'oth'}, inplace = True)
start = pd.to_datetime('1998-01-01')
rng = pd.date_range(start, periods=36, freq='3M')
index = pd.Index(rng)
df1 = df
df1 =df1.set_index(index)
df1.index.name = 'date'

freq = 'Q'

# %%
# Here we set the length of the train, test and validation set 
# We can generate the time series of desired size using following parameters
#there are total 36 data points in each time series 
total_len = len(df1)
train_len = total_len-test_len
split_val = train_len

purpose = np.array(df1.columns[1:5])
purpose_r = list(range(1,5))
states = np.array(df1.columns[5:33])
states_r = list(range(5,33))
city = np.array(df1.columns[33:len(df1.columns)])
city_r = list(range(33,len(df1.columns)))

level0 = [0]
levels = [level0, purpose_r,states_r,city_r]
bottom_level = city_r
levels1 = [purpose,states,city]

agg_length= 33

# Defining hierarchy

total = {'total': list(purpose)}
purpose_h  = {k: [v for v in states  if v.endswith(k)] for k in purpose}
state_h =  {k: [v for v in city  if v.startswith(k)] for k in states}

hierarchy ={**total,**purpose_h,**state_h}  #state_h

aac = list(hierarchy.keys())   # aggregated series 
aab= list(hierarchy.values())  # child nodes of each aggregated series 


'''Defining aggregation structure that will be needed while geeting the aggregayed 
forecast using child node forecasts'''
agg_structure1 = []
for i in range(len(aac)):
    for j in range(len(levels1)):
        if aab[i][0] in levels1[j]:
            abcd = []
            for l in aab[i]:
                abcd.append(list(levels1[j]).index(l))
            agg_structure1.append(abcd)


'''Adding bottom level to the aggregation structure'''
for i in range(len(bottom_level)):
    agg_structure1.append([i])

# %%
#defining the tree for hierarchy 
ht = HierarchyTree.from_nodes(nodes=hierarchy, df=df1)
#ht = HierarchyTree.from_nodes(nodes=hierarchy, df=df)
sum_mat, sum_mat_labels = hts.functions.to_sum_mat(ht) #sum_mat_lables are the all the nodes in the hierarchy
df2 = pd.DataFrame(columns = sum_mat_labels)
for col in sum_mat_labels:
    df2[col] = df1[col]
   
  
df2 = df1.iloc[: , :len(agg_structure1)]
df2['date'] =  df2.index
df =df2               #df and df2 are equal in lenght 
    
    
#splitting data as train and test
df = df.drop(df.columns[-1], axis=1)

df_test = df2[train_len: len(df2)] 

columns = df.columns
df_list = []        # origignal data in list format 
for i in columns:
    df_list.append(df2[i].to_list())
# %%


In [ ]:
from gluonts.dataset.common import ListDataset

In [ ]:
metadata = {
    'prediction_length': test_len,
    'freq': freq
}


train_data = [{"start": df.index[0], "target": df[i].values[:-metadata['prediction_length']]} for i in df.columns]
test_data = [{"start": df.index[0], "target": df[i].values} for i in df.columns]

train_ds = ListDataset(
    data_iter=train_data,
    freq=metadata['freq']
)

test_ds = ListDataset(
    data_iter=test_data,
    freq=metadata['freq']
)
device = torch.device("cuda:0")

In [ ]:
#metadata
prediction_length = test_len
context_length = test_len
num_samples = 24
device = "cuda"

In [ ]:
forecasts, tss = get_lag_llama_predictions(
    test_ds,
    prediction_length=prediction_length,
    num_samples=num_samples,
    context_length=context_length,
    device=device
)

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

# Iterate through the first 9 series, and plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
from gluonts.evaluation import Evaluator
quantile = [0.1, 0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9]

evaluator = Evaluator(quantiles=quantile)
agg_metrics, item_metrics = evaluator(tss, forecasts)
agg_metrics

In [ ]:
ckpt = torch.load("lag-llama.ckpt", map_location=device)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        # distr_output="neg_bin",
        # scaling="mean",
        nonnegative_pred_samples=True,
        aug_prob=0,
        lr=1e-2,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        time_feat=estimator_args["time_feat"],

        # rope_scaling={
        #     "type": "linear",
        #     "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        # },

        batch_size=32,
        num_parallel_samples=num_samples,
        trainer_kwargs = {"max_epochs": 150,}, # <- lightning trainer arguments
    )

In [ ]:
from gluonts.evaluation import make_evaluation_predictions

In [ ]:
predictor = estimator.train(train_ds, cache_data=True, shuffle_buffer_length=1000)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,
        predictor=predictor,
        num_samples=num_samples,
    )

In [ ]:
forecasts =  list(tqdm(forecast_it, desc="Forecasting batches"))
tss =  list(tqdm(ts_it,desc="Ground truth"))

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

# Iterate through the first 9 series, and plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=quantile)
agg_metrics, item_metrics = evaluator(tss, forecasts)

In [ ]:

def process_forecasts(forecasts):
    samples1 = [forecast.samples for forecast in forecasts]
    samples = np.transpose(samples1, (0, 2, 1))
    samples = np.array(samples)
    return samples

# Usage example:
samples = process_forecasts(forecasts)


In [ ]:
import numpy as np


a = num_samples
hist_i = a // metadata['prediction_length']

past_m = []   
past_std = []

for i in range(hist_i):
    try:
        train_data_h = [{"start": df.index[0], "target": df[i1].values[:-a ]} for i1 in df.columns]
        train_ds_h = ListDataset(data_iter=train_data_h, freq=metadata['freq'])
        forecast_it_h, ts_it_h = make_evaluation_predictions(
            dataset=train_ds_h,
            predictor=predictor,
            num_samples=num_samples)

        forecasts_h = list(tqdm(forecast_it_h, desc="Forecasting batches"))
        tss_h = list(tqdm(ts_it_h, desc="Ground truth"))
    
        samples11 = []
        for n in range(len(forecasts_h)):
            samples11.append(forecasts_h[n].samples)
            
        samples1 = np.transpose(samples11, (0, 2, 1))
        samples1 = samples1.tolist()
        samples1 = np.array(samples1)

        for k in range(len(samples1)):
            aa = np.mean(samples1[k], axis=1)
            ab = np.std(samples1[k], axis=1)
            
            if i <= 0:
                past_m.append(aa)
                past_std.append(ab)
            else:
                past_m[k] = np.hstack((past_m[k], aa))
                past_std[k] = np.hstack((past_std[k], ab))

        a = a - metadata['prediction_length']

    except Exception as e:
        print(f"An error occurred: {e}")
        # Handle the error gracefully, log it, and possibly retry or exit the loop


In [ ]:
'''Defining the standard residuals for the past data., taking mean and sd from past data samples 

     residuals = (actual-mean(predicted))/std(predicted)
     
 We want to store the ranks of these residuals to use in revised bottom-up forecasting. '''
res = []
for i in range(len(samples)):
    ab = []
    for j in range(num_samples):
        p = len(df[columns[0]])-num_samples-test_len
        aa = ((df[columns[i]][p+j]-past_m[i][j]))/past_std[i][j]
        ab.append(aa)
    res.append(ab)

In [ ]:
'''We need sorted samples to re-order accoeding to the ranks of the residuals'''

prob_forecasting = probabilistic_forecasting()

sorted_samples = prob_forecasting.sample_sorting_1(samples) # sample_sorting function is defined in forecasting_funtions.py file 


In [ ]:
#defining ranks for the residuals
ranks=[]
for i in range(len(res)):
    ranks.append(ss.rankdata(res[i]))

In [ ]:
reorderd_samples = prob_forecasting.sample_reordering_3(samples, ranks)

In [ ]:

BU_len = sum_mat.shape[1]       #len of the bottom series 
agg_len = len(sum_mat_labels)-BU_len  #length of the aggregated series
agg_samples_index = list(range(0, agg_len))  #indices of the aggregated series 
agg_series_ = columns[0:agg_len]           # names of the aggregated series 

bottom_series = prob_forecasting.getting_bottom_series(samples, sum_mat,sum_mat_labels)   #bottom level without reodering
bottom_series_r = prob_forecasting.getting_bottom_series(reorderd_samples,sum_mat,sum_mat_labels) #bottom series with rank reodering

aggre_series_numbers = agg_structure1     #aggregation structutre 


In [ ]:
# bottom series of original observation 
bottom_series_og = prob_forecasting.getting_bottom_series(df_list,sum_mat,sum_mat_labels) #function defined in forecasting_funtions.py file 


In [ ]:
# forecat using bottom-up method 
bottom_up_samples_ = prob_forecasting.bottom_up_forecast(columns,bottom_series,bottom_level,aggre_series_numbers,levels) #bottom_up_forecast function defined in forecasting_funtions.py file
bottom_up_samples_ = prob_forecasting.list_reversal(bottom_up_samples_) # list_reversal function defined in forecasting_funtions.py file

In [ ]:
# BU forecast with reodering, we reoder the forecasted samples according to the ranks of the past forecast residuals
revised_samples_ = prob_forecasting.bottom_up_revised_forecast(columns,bottom_series_r,bottom_level,aggre_series_numbers,levels,ranks)
revised_samples_ = prob_forecasting.list_reversal(revised_samples_)

In [ ]:
#Bottom up forecast using outlier reordering technique
bottom_up_forecast_h = prob_forecasting.bottom_up_forecast_hueristic(bottom_series, bottom_series_og,columns,bottom_level,aggre_series_numbers,levels,0.3)
bottom_up_samples_h = prob_forecasting.list_reversal(bottom_up_forecast_h)

In [ ]:
#mean forecast using all three methods 
mean_bottom_up_ = prob_forecasting.finding_mean(bottom_up_samples_)
mean_revised_forecast_ = prob_forecasting.finding_mean(revised_samples_)
mean_bottom_up_h = prob_forecasting.finding_mean(bottom_up_samples_h)

In [ ]:
from pandas import Period
# Define the start date
start_period = Period(df_test.index[0], freq)

In [ ]:
from gluonts.model.forecast import SampleForecast

bottom_up_samples_ = np.transpose(bottom_up_samples_,(0,2,1))

revised_samples_ = np.transpose(revised_samples_,(0,2,1))

bottom_up_samples_h = np.transpose(bottom_up_samples_h,(0,2,1))

# BU samples 
BU_forecast = [SampleForecast(samples=samples, start_date=start_period) for samples in bottom_up_samples_]


# revised samples
r_forecast = [SampleForecast(samples=samples, start_date=start_period) for samples in revised_samples_]



# HRA samples
h_forecast = [SampleForecast(samples=samples, start_date=start_period) for samples in bottom_up_samples_h]

loss_df = pd.DataFrame(columns=['Method', 'mean_absolute_QuantileLoss', 'mean_wQuantileLoss'])


In [ ]:
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, forecasts)
new_row = pd.Series({'Method': 'Base_forecast', 'mean_absolute_QuantileLoss': agg_metrics['mean_absolute_QuantileLoss'], 'mean_wQuantileLoss': agg_metrics['mean_wQuantileLoss']})
loss_df.loc[len(loss_df)] = new_row

In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, BU_forecast)
new_row = pd.Series({'Method': 'BU_forecast', 'mean_absolute_QuantileLoss': agg_metrics['mean_absolute_QuantileLoss'], 'mean_wQuantileLoss': agg_metrics['mean_wQuantileLoss']})
loss_df.loc[len(loss_df)] = new_row

In [ ]:
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, r_forecast)
new_row = pd.Series({'Method': 'r_forecast', 'mean_absolute_QuantileLoss': agg_metrics['mean_absolute_QuantileLoss'], 'mean_wQuantileLoss': agg_metrics['mean_wQuantileLoss']})
loss_df.loc[len(loss_df)] = new_row

In [ ]:
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, h_forecast)
new_row = pd.Series({'Method': 'h_forecast', 'mean_absolute_QuantileLoss': agg_metrics['mean_absolute_QuantileLoss'], 'mean_wQuantileLoss': agg_metrics['mean_wQuantileLoss']})
loss_df.loc[len(loss_df)] = new_row

In [ ]:
loss_df